In [ ]:
import os
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader, Subset
from torchvision.datasets import ImageFolder
import torch.optim as optim
import torch.nn as nn
import numpy as np
import pandas as pd

In [ ]:
# 设置数据路径
data_dir = 'D:\FDU\Course\DeepLearning\midterm\CUB_200_2011\CUB_200_2011'
images_dir = os.path.join(data_dir, 'images')

# 读取划分文件
split_data = pd.read_csv(os.path.join(data_dir, 'train_test_split.txt'), 
                         delim_whitespace=True, header=None, names=['id', 'train_flag'])

# 设置图像变换
transformations = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 加载全部图像
full_dataset = ImageFolder(root=images_dir, transform=transformations)

# 分割训练集和测试集
train_idx = split_data[split_data['train_flag'] == 1]['id'].values - 1
test_idx = split_data[split_data['train_flag'] == 0]['id'].values - 1

train_dataset = Subset(full_dataset, train_idx)
test_dataset = Subset(full_dataset, test_idx)

In [ ]:
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

In [ ]:
def train(model, loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    for inputs, labels in loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    return running_loss / len(loader.dataset)

def validate(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            _, preds = torch.max(outputs, 1)
            correct += torch.sum(preds == labels.data)
            running_loss += loss.item() * inputs.size(0)
    accuracy = correct.double() / len(loader.dataset)
    return running_loss / len(loader.dataset), accuracy.item()


In [ ]:
# 加载预训练的ResNet-18模型
model = resnet18(pretrained=True)

# 修改输出层
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 200)


# 定义设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 为新的输出层设置高学习率，其他层设置低学习率
params_to_update = []
params_names = []

# 只对非全连接层参数应用低学习率
for name, param in model.named_parameters():
    if param.requires_grad == True:
        if name not in ['fc.weight', 'fc.bias']:
            params_to_update.append(param)
            params_names.append(name)


In [ ]:
for learning_rate in [0.01,0.05,0.1]:
    for e in [5,10,20]:
        # 初始化TensorBoard
        writer = SummaryWriter('runs/birds_experiment_'+str(e)+'_'+str(learning_rate))

        # 使用两个参数组
        optimizer = optim.SGD([
            {'params': model.fc.parameters(), 'lr': learning_rate},  # 高学习率仅适用于全连接层
            {'params': params_to_update, 'lr': 0.001}       # 低学习率适用于其余层
        ], momentum=0.9)


        # 损失函数
        criterion = nn.CrossEntropyLoss()

        epochs = e
        for epoch in range(epochs):
            train_loss = train(model, train_loader, optimizer, criterion, device)
            test_loss, test_accuracy = validate(model, test_loader, criterion, device)
            print(f'Epoch {epoch+1}, Train Loss: {train_loss}, Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')
            writer.add_scalar('Loss/Train', train_loss, epoch)
            writer.add_scalar('Loss/Test', test_loss, epoch)
            writer.add_scalar('Accuracy/Test', test_accuracy, epoch)

        writer.close()
